# Analysis Pipeline for Intracranial Data

<!-- ### Triggers/Events
Using photo triggers

### Preprocessing
TODO: ADD ouline of preprocessing steps for iEEG data

### MNE python
Working on the MNE library to recreate the same matlab funcitons: 
Location:
https://mne.tools/stable/auto_tutorials/intro/10_overview.html#sphx-glr-auto-tutorials-intro-10-overview-py -->
## STEPS:
##### 1. run the IMPORT AND LOAD DATA section.
updave the following variables:
```data_raw_file=
data_path = 
trigger_channel=
```
##### 2. fill in the values extracted from the plot from step 1 for following variables:
```
t_start = 
t_end= 
threshold_val=
```


Let's go

In [ ]:
import os
import numpy as np
import mne
from mne.utils import _TempDir
import matplotlib.pyplot as plt
import pyqtgraph as pg
import pyqtgraph as plotWidget
%matplotlib qt
from pd_parser.parse_pd import _read_raw, _to_tsv

# DEFINE PATHS
data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066'
script_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/Scripts'
data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf')

# LOAD TRIGGER DATA 
trigger_ch = mne.io.read_raw_edf(data_raw_file,exclude=['ECG+','ECG-','MKR4+','Xe2', 'Xe3', 'Xe4', 'Xe5', 'Xe6'])
print('*   Trigger channel length: ',trigger_ch.__len__)


picks = mne.pick_channels_regexp(trigger_ch.info.ch_names, regexp='Xe1')
fig = plt.figure(figsize=(100, 2))
trigger_ch2 = trigger_ch.get_data(picks=picks)
trigger_ch2 = trigger_ch2[0].T
time_s = np.linspace(0,len(trigger_ch2),num=len(trigger_ch2))

pg.plot(time_s,trigger_ch2)
plt.close(fig)


# VISUALIZE
fig = plt.figure(figsize=(100, 2))
print(np.shape(trigger_ch2))
import pyqtgraph as pg
import pyqtgraph as plotWidget
pg.plot(time_s,trigger_ch2)
plt.close(fig)


In [ ]:
from get_trigger_indexes_photodiode import get_trigger_indexes_photodiode
# t_onset_s = input("Enter cut start time: ")
# t_offset_s = input("Enter cut end time: ")
t_onset_s= 1000000
t_offset_s= 1900000
index_onset, index_offset, trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.85)


### Save The Data

In [26]:
with open(os.path.join(data_path, "TESTtriggers3_PAT3066_visualCatLoc.txt"), "w") as textfile:
    textfile.write('onsets;offsets\n')
    XY = [i for i in zip(index_onset, index_offset)]
    for x, y in XY:
        textfile.write(str(x)+';'+str(y))
        textfile.write('\n')
    # for x, y in index_onset,index_offset:
    #     textfile.write(str(x)+';'+str(y)+';')
    # # textfile.write('onsets;')
    # [textfile.write(str(x)+';') for x in index_onset]
    # # textfile.write('\noffsets;')
    # [textfile.write(str(x)+';') for x in index_offset]
    textfile.close()

In [21]:


print(XY)

30446;31098;
32724;33588;
35179;36249;
37876;38941;
40530;41190;
42778;43440;
45064;45897;
47520;47980;
49565;50468;
52028;52691;
54312;55177;
56769;57633;
59226;60291;
61917;62342;
63967;65037;
66625;67287;
68875;69947;
71572;72637;
74263;74922;
76547;77381;
78973;79433;
81050;82121;
83712;84780;
86410;87270;
88866;89728;
91319;91979;
93607;94270;
95852;96921;
98509;99172;
100757;101826;
103452;103908;
105499;106364;
107958;108416;
110007;110873;
112499;112958;
114543;115205;
116791;117251;
118871;119333;
120919;121583;
123203;124067;
125690;126557;
128147;129216;
130807;131873;
133471;134334;
135965;136828;
138416;139078;
140697;141157;
142742;143609;
145198;145862;
147450;148113;
149732;150598;
152190;153053;
154646;155138;
156690;157386;
158974;159400;
161021;161483;
163072;164141;
165731;166661;
168249;169115;
170706;171571;
173162;173824;
175412;176278;
177902;178968;
180563;181426;
183014;183675;
185297;185758;
187340;188445;
190005;190463;
192084;192713;
194333;194997;
196586;1

### FINISHED HERE: 
##### Everything below is draft

## IMPORT AND LOAD DATA

In [ ]:
# IMPORTS: 
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import mne
import scipy
from scipy import signal
from mne.utils import _TempDir
import matplotlib.pyplot as plt
%matplotlib qt
from pd_parser.parse_pd import _read_raw, _to_tsv

# out_dir = _TempDir()
# print(f'Data location.... : {out_dir}')
# DATA LOCATION:
data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066'
# data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/S20'
script_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/Scripts'


# LOAD DATA 
# data_raw_file = os.path.join(data_path, 'S20_20181217_014956.mff')
# data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_695997_Anon_FLM.edf')
# fig = plt.figure(figsize=(100, 2))

data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf')
trigger_ch = mne.io.read_raw_edf(data_raw_file,exclude=['ECG+','ECG-','MKR4+','Xe2', 'Xe3', 'Xe4', 'Xe5', 'Xe6'])
print(trigger_ch.__len__)
picks = mne.pick_channels_regexp(trigger_ch.info.ch_names, regexp='Xe1')
# picks = mne.pick_channels_regexp(trigger_ch.info.ch_names, regexp='')
# trigger_ch.plot(order=picks,n_channels=203, event_color='cyan', scalings=3700, duration=1200, show_options=True)

t_onset_s= 1000000
t_offset_s= 1900000
trigger_ch = trigger_ch.get_data(picks=picks,start=t_onset_s,stop=t_offset_s)
print(np.shape(trigger_ch[0]))
trigger_ch2 = trigger_ch[0] #np.array(trigger_ch).T

time_s = np.linspace(0,len(trigger_ch[0]),num=len(trigger_ch[0]))
del trigger_ch

%matplotlib qt
fig = plt.figure(figsize=(100, 2))
print(np.shape(trigger_ch2))
import pyqtgraph as pg
import pyqtgraph as plotWidget
pg.plot(time_s,trigger_ch2)
plt.close(fig)


# plt.close(fig)
# del trigger_ch
# # IMPORTS: 
# from sklearn.preprocessing import StandardScaler
# import os
# import numpy as np
# import mne
# import scipy
# from scipy import signal
# from mne.utils import _TempDir
# import matplotlib.pyplot as plt
# %matplotlib qt
# # %matplotlib qt
# from pd_parser.parse_pd import _read_trigger_ch, _to_tsv

# # out_dir = _TempDir()
# # print(f'Data location.... : {out_dir}')
# # DATA LOCATION:
# data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066'
# txt_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066'

# # data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/S20'
# script_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/Scripts'



# # LOAD DATA 
# # data_raw_file = os.path.join(data_path, 'S20_20181217_014956.mff')
# # data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_695997_Anon_FLM.edf')
# data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf')
# raw = mne.io.read_raw_edf(data_raw_file,exclude=['ECG+','ECG-','MKR4+','Xe2', 'Xe3', 'Xe4', 'Xe5', 'Xe6'])
# print(raw.__len__)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='Xe1')
# # picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='')
# raw.plot(order=picks,n_channels=203, event_color='cyan', scalings='auto', duration=300)
# raw.get_data(picks=picks)
# raw2 = raw[-1][0]
# raw2 = np.array(raw2[0])

# del raw

## NORMALIZE TRIGGER DATA: PHOTODIODE

In [ ]:
# NORMALIZE
# def normalized(data):
#     return (data - np.min(data)) / (np.max(data) - np.min(data))
# t_onset_s= 275
# t_offset_s= 1620
# print(time_s)
# trigger_ch2 = normalized(trigger_ch2)



## VISUALIZE TRIGGER DATA: PHOTODIODE

In [ ]:
# # VISUALIZE
# %matplotlib qt
# fig = plt.figure(figsize=(100, 2))
# print(np.shape(trigger_ch2))
# import pyqtgraph as pg
# import pyqtgraph as plotWidget
# pg.plot(time_s,trigger_ch2)
# plt.close(fig)


## PREPROCESSING TRIGGER DATA TO GET EVENTS: PHOTODIODE

In [ ]:
from savitzky_golay import savitzky_golay
from get_trigger_indexes_photodiode import get_trigger_indexes_photodiode
t_onset_s= 1000000
t_offset_s= 1900000
index_onset, index_offset,trigger_ch_new,_= get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.9,do_plot=True)


#### SAMPLE

In [ ]:
## SMOOTH
from savitzky_golay import savitzky_golay
from get_trigger_indexes_photodiode import get_trigger_indexes_photodiode
t_onset_s= 0
t_offset_s= -1
index_onset, index_offset = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.5)

# print(raw2)
# temp = savitzky_golay(trigger_ch, 11, 3) # window size 51, polynomial order 3
t_onset_s= 275
t_offset_s= 1620

# print(len(index_onset), len(index_offset))
# time2 = time_s[10000:50000]
# temp2 = temp[0:40000]
# temp3 = np.diff(temp[0:40000])
# plotWidget = pg.plot(title="Trigger raw + diff: sample")

# plotWidget.plot(time2,temp2,pen='blue')
# plotWidget.plot(time2[0:-1],temp3,pen='white')


#### FULL SIZE

In [ ]:
# time2 = time_s
# temp2 = time2
# temp3 = np.diff(time2)
# # plotWidget = pg.plot(title="Trigger raw + diff")

# plotWidget.plot(time2,temp2,pen='blue')
# plotWidget.plot(time2[0:-1],temp3,pen='white')

# # picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='Xe1')
# # raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False) 
# # picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='')
# # raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False)


#### FIND MAXIMA MINIMA

In [ ]:
import numpy as np
from scipy.signal import argrelextrema

# temp3 = normalized(temp3)

# plotWidget = pg.plot(title="Trigger Max and min convolved V5")

# # for local maxima
# indexes = scipy.signal.argrelextrema(temp3, np.greater,order=200)[0]
# print(len(temp3[scipy.signal.argrelextrema(temp3, np.greater, order=200)[0][:]]))
# print(len(indexes[:]))
# plotWidget.plot(time2[0:-1],temp3,pen='white')
# plotWidget.plot(time2[0:-1],temp3,pen='white')
# plotWidget.plot(indexes,temp3[indexes],pen=None, symbol='o')

# for local minima



## ONSET TRIGGERS: 

In [ ]:
# Filtered plots
# raw_original = raw.copy()

# index_final_onset,_ = scipy.signal.find_peaks(temp3, height=0.8*temp3.max())
# print(index_final_onset)

# plotWidget = pg.plot(title="Trigger Max and min convolved V4")

# for local maxima
# print(len(temp[index_final_onset]))
# print(len(index_final_onset[:]))
# plotWidget.plot(time2[0:-1],temp3,pen='white')
# plotWidget.plot(index_final_onset,temp3[index_final_onset],pen=None, symbol='o')

OFFSET TRIGGERS

In [ ]:
# Filtered plots
# # raw_original = raw.copy()


# index_final_offset,_ = scipy.signal.find_peaks(-1*temp3, height=-0.2)
# print(index_final_offset)

# plotWidget = pg.plot(title="Trigger Max and min convolved V4")

# # for local maxima
# print(len(temp[index_final_offset]))
# print(len(index_final_offset[:]))
# plotWidget.plot(time2[0:-1],temp3,pen='white')
# plotWidget.plot(index_final_offset,temp3[index_final_offset],pen=None, symbol='o')
# plotWidget.plot(index_final_onset,temp3[index_final_onset],pen=None, symbol='x')

## ALL TOGETHER

In [ ]:
# Filtered plots
# raw_original = raw.copy()


plotWidget = pg.plot(title="Trigger Max and min convolved V6")

# for local maxima
# print(time_s)
# print(len(time_s[index_offset]))
# print(len(index_offset[:]))
# plotWidget.plot(time2,time_s,pen='blue')
# plotWidget.plot(time2[0:-1],temp3,pen='white')
# plotWidget.plot(time_s,trigger_ch2)
# plotWidget.plot(index_offset,time_s[index_offset],pen=None, symbol='o')
# plotWidget.plot(index_onset,time_s[index_onset],pen=None, symbol='x')
plotWidget.plot(time_s,trigger_ch2)
plotWidget.plot(index_offset,trigger_ch2[index_offset],pen=None, symbol='o')
plotWidget.plot(index_onset,trigger_ch2[index_onset],pen=None, symbol='x')


# # Filtered plots
# # raw_original = raw.copy()


# index_final_offset,_ = scipy.signal.find_peaks(-1*temp3, height=-0.2)
# print(index_final_offset)

# plotWidget = pg.plot(title="Trigger Max and min convolved V6")

# # for local maxima
# print(len(temp[index_final_offset]))
# print(len(index_final_offset[:]))
# plotWidget.plot(time2,temp2,pen='blue')
# plotWidget.plot(time2[0:-1],temp3,pen='white')
# plotWidget.plot(index_final_offset,temp3[index_final_offset],pen=None, symbol='o')
# plotWidget.plot(index_final_onset,temp3[index_final_onset],pen=None, symbol='x')


## EXPORT TEXT FILE

In [ ]:
data.preload_data()
data.filter(l_freq=75, h_freq=175, picks=picks)
data.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False)

data.notch_filter(np.arange(60, 241, 60), picks=picks, filter_length='auto',phase='zero')
data.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False)


# raw.preload_data()
# raw.filter(l_freq=0.15, h_freq=None, picks=picks)
# raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False)

# raw.notch_filter(np.arange(60, 241, 60), picks=picks, filter_length='auto',phase='zero')
# raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False)


In [ ]:
# # FIND TRIGGERS FROM PHOTODIODE
# # print(len(raw.get_data().T))
# %matplotlib qt
# pd_parser.find_pd_params(raw, pd_ch_names=['Xe1'])
# pd_parser.parse_pd(raw, pd_ch_names=['Xe1'], max_len=15.83, zscore=-2.26,overwrite=True)
# # print(Xe1_params)
# # print(xe1_params2)

In [ ]:
# # Some information about the DATASET: 
# print(raw.info)
# infodata = raw.info

# ## FURTHER INFORMATION
# # print(raw.info.ch_names)
# # print(raw.annotations)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='')
# # print(picks)
# # raw.plot(order=picks, n_channels=len(picks), duration=5, scalings=1000, start=120)
# raw2 = raw.copy()
# events = mne.find_events(raw2, stim_channel='Xe1', initial_event=False,uint_cast=True, min_duration=3/raw.info['sfreq'], )
# raw2.plot(order=picks, events=events, n_channels=203, duration=20, event_color='cyan', scalings=500, start=200)
# out_dir = _TempDir()


## FILTERING

In [ ]:
# # raw2.plot(order=picks, n_channels=203, duration=20, event_color='cyan', scalings=1000, start=190)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='Xe1')

# raw2.plot(order=picks, n_channels=1, duration=500, event_color='cyan', scalings=5000, start=190)

# raw2.apply_hilbert(picks=picks)
# raw2.plot(order=picks, n_channels=1, duration=500, event_color='cyan', scalings=5000, start=190)


In [ ]:
# FILTERING


In [ ]:

# # raw.plot(duration=60)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='Xe')
# raw.plot(order=picks, n_channels=len(picks), duration=1600, scalings=3000, start=120)

In [ ]:
# raw2 = raw.copy()
# raw2.info['bads'] = []
# picks = mne.pick_channels_regexp(raw2.info.ch_names, regexp='S')
# print(picks)
# raw2.plot(order=picks, n_channels=len(picks), duration=2)


In [ ]:
# raw2 = raw.copy()
# raw2.info['bads'] = []
# picks = mne.pick_channels_regexp(raw2.info.ch_names, regexp='ST')
# print(picks)
# events = mne.find_events(raw2, stim_channel='Xe1', initial_event=True,uint_cast=True)

# raw2.plot(order=picks, n_channels=len(picks), duration=5, event_color='cyan', scalings=0.1)
# print(events)
# epochs = mne.Epochs(raw2, events=events)['2'].average().plot()

### Independend Component Analysis: 
```
[CleanData, trials, BadIdx] = reject_artefact(EventData);
save TrialIndex trials
save BadChannelIndex BadIdx

popout_ica(CleanData);

[cleanBEEPData, ~ , ~] = reject_artefact(BEEPdata);

popout_ica(cleanBEEPData);
```



In [ ]:
# # set up and fit the ICA
# ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
# ica.fit(raw)
# temporary = os.path.join(data_path, 'BadChannelIndex.mat')
# ica.exclude = scipy.io.loadmat(temporary)  # [1, 2]  # details on how we picked these are omitted here

# ica.plot_properties(raw, picks=ica.exclude)

In [ ]:
# orig_raw = raw.copy()
# raw.load_data()
# ica.apply(raw)

# # identify channels
# temporary = os.path.join(data_path, 'Params.mat')
# params = scipy.io.loadmat(temporary) 

# # show some frontal channels to clearly illustrate the artifact removal
# chs =  params.channels # ['e1','e2']
# chan_idxs = [raw.ch_names.index(ch) for ch in chs]
# orig_raw.plot(order=chan_idxs, start=12, duration=4)
# raw.plot(order=chan_idxs, start=12, duration=4)

### Triggers: 
```
events = mne.find_events(raw, stim_channel='STI 014')
print(events[:5])  # show the first 5
```

In [ ]:
# events = mne.find_events(raw, stim_channel='STI 014')
# print(events[:5])  # show the first 5

# # event_dict = {'auditory/left': 1, 'auditory/right': 2, 'visual/left': 3,
# #               'visual/right': 4, 'smiley': 5, 'buttonpress': 32} 
# event_dict = params.triggers

# fig = mne.viz.plot_events(events, event_id=event_dict, sfreq=raw.info['sfreq'],
#                           first_samp=raw.first_samp)